### Use Google Earth Engine and `geemap` to get repeat images of the Pommeroye catchment

(See https://github.com/giswqs)

In [1]:
import os

# Instal geemap package
import subprocess
import geemap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

The Pommeroy catchment is at 50°28'19.1"N and 2°03'40.5"E, or in decimal:

In [2]:
lat = 50 + (28/60) + 0.01*(19.1/60)
lon = 2 + (3/60) + 0.01*(40.5/60)
print('lat: {}°N, lon: {}°E'.format(lat, lon))

lat: 50.46985°N, lon: 2.0567499999999996°E


In [10]:
Map = geemap.Map(center=[lat, lon], zoom=16)
Map

Map(center=[50.46985, 2.0567499999999996], controls=(WidgetControl(options=['position'], widget=HBox(children=…

### Landsat images

Add Landsat 8 images (as done here https://github.com/giswqs/earthengine-py-notebooks/blob/master/GetStarted/03_finding_images.ipynb)

In [4]:
# Add Earth Engine dataset
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1')

point = ee.Geometry.Point(lon, lat)
start = ee.Date('2018-01-01')
finish = ee.Date('2018-06-30')

filteredCollection = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
    .filterBounds(point) \
    .filterDate(start, finish) \
    .sort('CLOUD_COVER', True)

first = filteredCollection.first()
# Define visualization parameters in an object literal.
vizParams = {'bands': ['B5', 'B4', 'B3'],
             'min': 5000, 'max': 15000, 'gamma': 1.3}
Map.addLayer(first, vizParams, 'Landsat 8 image')

Make a timelaps of Landsat 8 using `geemap` (https://gist.github.com/giswqs/bab47213f0cbd3d1073ea47b01331c62)

In [5]:
Map.setCenter(lon, lat, 16)
Map.add_landsat_ts_gif(roi=ee.Geometry.Rectangle([lon-0.1, lat-0.05, lon+0.1, lat+0.05]),
                       label='Pommeyore, Northern France',
                       bands=['SWIR1', 'NIR', 'Red'],
                       nd_bands=['Green', 'SWIR1'],
                       nd_palette=['black', 'blue'],
                       start_year=2016,
                       start_date='01-01',
                       end_year=2019,
                       end_date='12-31',
                       frames_per_second=1)

Generating URL...
Please wait ...
The GIF image has been saved to: /work/armitagj/Downloads/landsat_ts_nmp.gif
Generating URL...
Please wait ...
The GIF image has been saved to: /work/armitagj/Downloads/landsat_ts_nmp_nd.gif
Adding animated text to GIF ...
Adding GIF to the map ...
The timelapse has been added to the map.


### Sentinel-2 images

A wee hack to try and do the same thing as `add_landsat_ts_gif` but for Sentinel-2 images, as there is a greater repeat for this satellite (I think). Start off with the existing function:

In [6]:
collection = geemap.sentinel2_timeseries(roi=ee.Geometry.Rectangle([lon-0.1, lat-0.05, lon+0.1, lat+0.05]),
                                         start_year=2016,
                                         end_year=2019,
                                         start_date='01-01',
                                         end_date='12-31')

Then export the image (https://github.com/giswqs/geemap/blob/master/examples/notebooks/11_export_image.ipynb)

In [7]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')

In [8]:
print(collection.aggregate_array('system:index').getInfo())

['0', '1', '2', '3']


In [9]:
geemap.ee_export_image_collection(collection, out_dir=out_dir)

Total number of images: 4

Exporting 1/4: 0.tif
Generating URL ...
Please wait ...
Data downloaded to /work/armitagj/Downloads/0.tif


Exporting 2/4: 1.tif
Generating URL ...
Please wait ...
Data downloaded to /work/armitagj/Downloads/1.tif


Exporting 3/4: 2.tif
Generating URL ...
Please wait ...
Data downloaded to /work/armitagj/Downloads/2.tif


Exporting 4/4: 3.tif
Generating URL ...
Please wait ...
Data downloaded to /work/armitagj/Downloads/3.tif




In [ ]:
first = collection.first()
# Define visualization parameters in an object literal.
vizParams = {'bands': ['Blue', 'Green', 'Red'],
             'min': 0, 'max': 5000, 'gamma': 1.3}
Map.addLayer(first, vizParams, 'Sentinel-2 image')

Now have a look inside and build my own monthly image set:
(functions from https://giswqs.github.io/geemap/eefolium/#geemap.eefolium.sentinel2_timeseries)

In [11]:
# Get Sentinel 2 collections, both Level-1C (top of atmophere) and Level-2A (surface reflectance)
MSILCcol = ee.ImageCollection('COPERNICUS/S2')
MSI2Acol = ee.ImageCollection('COPERNICUS/S2_SR')
MSI2Cloudcol = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")

In [12]:
# Define a collection filter by date, bounds, and quality.
def colFilter(col, roi, start_date, end_date):
    return(col
           .filterBounds(roi)
           .filterDate(start_date, end_date))

In [13]:
 # Function to get and rename bands of interest from MSI
def renameMSI(img):
    return(img.select(
        ['B2', 'B3', 'B4', 'B5', 'B6', 'B7',
            'B8', 'B8A', 'B11', 'B12', 'QA60'],
        ['Blue', 'Green', 'Red', 'Red Edge 1', 'Red Edge 2', 'Red Edge 3', 'NIR', 'Red Edge 4',
            'SWIR1', 'SWIR2', 'QA60']))

# Add NBR for LandTrendr segmentation.
def calcNbr(img):
    return(img.addBands(img.normalizedDifference(['NIR', 'SWIR2'])
                        .multiply(-10000).rename('NBR')).int16())

# Define function to mask out clouds and cloud shadows in images.
# Use CFmask band included in USGS Landsat SR image product.
def fmask(img):
    cloudOpaqueBitMask = 1 << 10
    cloudCirrusBitMask = 1 << 11
    qa = img.select('QA60')
    mask = qa.bitwiseAnd(cloudOpaqueBitMask).eq(0) \
        .And(qa.bitwiseAnd(cloudCirrusBitMask).eq(0))
    return(img.updateMask(mask))

# Define function to prepare MSI images.
def prepMSI(img):
    orig = img
    img = renameMSI(img)
    img = fmask(img)
    return(ee.Image(img.copyProperties(orig, orig.propertyNames()))
           .resample('bicubic'))

In [14]:
# Make a dummy image for missing years.
bandNames = ee.List(['Blue', 'Green', 'Red', 'Red Edge 1',
                     'Red Edge 2', 'Red Edge 3', 'NIR',
                     'Red Edge 4', 'SWIR1', 'SWIR2', 'QA60'])
fillerValues = ee.List.repeat(0, bandNames.size())
dummyImg = ee.Image.constant(fillerValues).rename(bandNames) \
    .selfMask().int16()

Make a list of dates for the months

In [15]:
months = geemap.date_sequence('2016-01-01', '2019-01-01', 'month')
roi=ee.Geometry.Rectangle([lon-0.1, lat-0.05, lon+0.1, lat+0.05])

In [22]:
# Get monthly median collection.
def getMonthlyComp(m):
    #print(m)
    startDate = ee.Date(m)
    endDate = startDate.advance(ee.Number(4), 'month')  # I find that a 4 monthly average gets rid of clouds...

    # Filter collections and prepare them for merging.
    MSILCcoly = colFilter(MSILCcol, roi, startDate, endDate).map(prepMSI)
    MSI2Acoly = colFilter(MSI2Acol, roi, startDate, endDate).map(prepMSI)

    # Merge the collections.
    col = MSILCcoly.merge(MSI2Acoly)

    yearImg = col.median()
    print('yearImg: {}'.format(type(yearImg)))
    nBands = yearImg.bandNames().size()
    yearImg = ee.Image(ee.Algorithms.If(
        nBands,
        yearImg,
        dummyImg))
    print('yearImg at end: {}'.format(type(yearImg)))
    return(calcNbr(yearImg)
           .set({'year': m, 'system:time_start': startDate.millis(), 'nBands': nBands}))

In [23]:
imgList = months.map(getMonthlyComp)

yearImg: <class 'ee.image.Image'>
yearImg at end: <class 'ee.image.Image'>
yearImg: <class 'ee.image.Image'>
yearImg at end: <class 'ee.image.Image'>


In [24]:
# Convert image composite list to collection
imgCol = ee.ImageCollection.fromImages(imgList)
imgCol = imgCol.map(lambda img: img.clip(roi))

In [25]:
print(imgCol.aggregate_array('system:index').getInfo())

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36']


Check...

In [ ]:
first = imgCol.first()
# Define visualization parameters in an object literal.
vizParams = {'bands': ['Blue', 'Green', 'Red'],
             'min': 0, 'max': 3000, 'gamma': 1.3}
Map.addLayer(first, vizParams, 'Sentinel-2 image')

In [ ]:
#Oh my God

Put monthly collection into a animated gif (https://github.com/giswqs/geemap/blob/master/examples/notebooks/16_add_animated_text.ipynb)

In [26]:
# Define arguments for animation function parameters.
videoArgs = {
  'dimensions': 768,
  'region': roi,
  'framesPerSecond': 1,
  'crs': 'EPSG:3857',
  'bands': ['Blue', 'Green', 'Red'],
  'min': 0,
  'max': 5000,
  'gamma': 1.3
}

In [27]:
saved_gif = os.path.join(os.path.expanduser('~'), 'Downloads/pommeyore.gif')
geemap.download_ee_video(imgCol, videoArgs, saved_gif)

Generating URL...
Please wait ...
The GIF image has been saved to: /work/armitagj/Downloads/pommeyore.gif


In [ ]:
# To do: add text to gif...

### Alternative cloud mask

(This does not work... yet)

https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_CLOUD_PROBABILITY

In [ ]:
# Alternative function to mask clouds
# https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_CLOUD_PROBABILITY
def maskClouds(img):
    MAX_CLOUD_PROBABILITY = 65
    clouds = ee.Image(img.get('cloud_mask')).select('probability')
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    return(img.updateMask(isNotCloud))

In [ ]:
# Define function to prepare MSI images.
def new_prepMSI(img):
    orig = img
    img = renameMSI(img)
    return(ee.Image(img.copyProperties(orig, orig.propertyNames()))
           .resample('bicubic'))

Javascript:
```
var s2Sr = ee.ImageCollection('COPERNICUS/S2_SR');
var s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY');

var START_DATE = ee.Date('2019-01-01');
var END_DATE = ee.Date('2019-03-01');
var MAX_CLOUD_PROBABILITY = 65;
var region =
    ee.Geometry.Rectangle({coords: [-76.5, 2.0, -74, 4.0], geodesic: false});
Map.centerObject(region, 12);

function maskClouds(img) {
  var clouds = ee.Image(img.get('cloud_mask')).select('probability');
  var isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY);
  return img.updateMask(isNotCloud);
}

// The masks for the 10m bands sometimes do not exclude bad data at
// scene edges, so we apply masks from the 20m and 60m bands as well.
// Example asset that needs this operation:
// COPERNICUS/S2_CLOUD_PROBABILITY/20190301T000239_20190301T000238_T55GDP
function maskEdges(s2_img) {
  return s2_img.updateMask(
      s2_img.select('B8A').mask().updateMask(s2_img.select('B9').mask()));
}

// Filter input collections by desired data range and region.
var criteria = ee.Filter.and(
    ee.Filter.bounds(region), ee.Filter.date(START_DATE, END_DATE));
s2Sr = s2Sr.filter(criteria).map(maskEdges);
s2Clouds = s2Clouds.filter(criteria);

// Join S2 SR with cloud probability dataset to add cloud mask.
var s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply({
  primary: s2Sr,
  secondary: s2Clouds,
  condition:
      ee.Filter.equals({leftField: 'system:index', rightField: 'system:index'})
});

var s2CloudMasked =
    ee.ImageCollection(s2SrWithCloudMask).map(maskClouds).median();
var rgbVis = {min: 0, max: 3000, bands: ['B4', 'B3', 'B2']};

Map.addLayer(
    s2CloudMasked, rgbVis, 'S2 SR masked at ' + MAX_CLOUD_PROBABILITY + '%',
    true);
```

In [ ]:
def getMonthlywithCloudMask(m):
    startDate = ee.Date(m)
    endDate = startDate.advance(ee.Number(4), 'month')

    # Filter collections and prepare them for merging.
    MSILCcoly = colFilter(MSILCcol, roi, startDate, endDate).map(prepMSI)
    MSI2Acoly = colFilter(MSI2Acol, roi, startDate, endDate).map(prepMSI)
    # MSI2Acloudcoly = colFilter(MSI2Cloudcol, roi, startDate, endDate).map(new_prepMSI)
    print('MSI2Acoly: {}'.format(type(MSI2Acoly)))

    # Merge the collections.
    col = MSILCcoly.merge(MSI2Acoly)
    print('col: {}'.format(type(col)))
    
    # Join MSI with cloud probability dataset to add cloud mask.
    # colWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(primary=col, secondary=MSI2Acloudcoly,
    #     condition = ee.Filter.equals(leftField='system:index', rightField='system:index'))
    # print('colWithCloudMask: {}'.format(type(colWithCloudMask)))
    
    yearImg = col.median()
    print('yearImg: {}'.format(type(yearImg)))
    
    # yearImg = col.median()
    # print('yearImg: {}'.format(type(yearImg)))
    
    nBands = yearImg.bandNames().size()
    yearImg = ee.Image(ee.Algorithms.If(
        nBands,
        yearImg,
        dummyImg))
    print('yearImg at end: {}'.format(type(yearImg)))
    return(calcNbr(yearImg)
           .set({'year': m, 'system:time_start': startDate.millis(), 'nBands': nBands}))

In [ ]:
imgList = months.map(getMonthlywithCloudMask)

In [ ]:
# Convert image composite list to collection
imgCol = ee.ImageCollection.fromImages(imgList)
imgCol = imgCol.map(lambda img: img.clip(roi))

In [ ]:
print(imgCol.aggregate_array('system:index').getInfo())

In [ ]:
Map = geemap.Map(center=[lat, lon], zoom=16)
Map

In [ ]:
first = imgCol.first()
# Define visualization parameters in an object literal.
vizParams = {'bands': ['Blue', 'Green', 'Red'],
             'min': 0, 'max': 3000, 'gamma': 1.3}
Map.addLayer(first, vizParams, 'Sentinel-2 image')

In [ ]:
saved_gif = os.path.join(os.path.expanduser('~'), 'Downloads/pommeyore_4month_median.gif')
geemap.download_ee_video(imgCol, videoArgs, saved_gif)